# Interactive Time Series Visualization Assignment

In [1]:
import pandas as pd
import plotly.express as px
from ipywidgets import interact

### Load the Online Retail.xlsx Excel file into a Pandas dataframe.

In [2]:
data = pd.read_excel('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Time%20Series%20Analysis/Time%20Series%20-%20Day%202/Online%20Retail.xlsx')

In [3]:
data.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
0,536365,2010-12-01 08:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
1,536373,2010-12-01 09:02:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
2,536375,2010-12-01 09:32:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
3,536390,2010-12-01 10:19:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.2,17511,United Kingdom
4,536394,2010-12-01 10:39:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13408,United Kingdom


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396034 entries, 0 to 396033
Data columns (total 9 columns):
InvoiceNo      396034 non-null int64
InvoiceDate    396034 non-null datetime64[ns]
StockCode      396034 non-null object
Description    396034 non-null object
Quantity       396034 non-null int64
UnitPrice      396034 non-null float64
Revenue        396034 non-null float64
CustomerID     396034 non-null int64
Country        396034 non-null object
dtypes: datetime64[ns](1), float64(2), int64(3), object(3)
memory usage: 27.2+ MB


### Create an interactive histogram to explore the distributions of daily (by date) Quantity, UnitPrice, and Revenue. Include a dropdown widget where you can switch between the three fields.

**Hint:** You will need to create a Date field and then aggregate on it. The Quantity and Revenue fields should be summed and the UnitPrice field should be averaged (mean).

In [5]:
data['date'] = data['InvoiceDate'].dt.date

aggs = {'Quantity': 'sum', 'Revenue': 'sum', 'UnitPrice': 'mean'}

daily_data = data.groupby('date', as_index=False).agg(aggs)#.reset_index()

daily_data.head()

,date,Quantity,Revenue,UnitPrice
0,2010-12-01,24207,46219.29,3.109995
1,2010-12-02,31140,47283.53,3.184610
2,2010-12-03,11822,23576.01,3.166670
3,2010-12-05,16370,31315.64,2.840382
4,2010-12-06,16267,31014.21,2.878689


In [6]:
def ihistogram(df, field, bins=None, title=''):
    fig = px.histogram(df, x=field, title=title, nbins=bins,
                       template='none').update(layout=dict(title=dict(x=0.5)))

    fig.update_yaxes(title_text='Number of Records')
    fig.update_traces(marker_color='lightskyblue',
                      marker_line_color='black', 
                      marker_line_width=1)

    fig.show()

In [15]:
fields = ['Quantity', 'UnitPrice', 'Revenue']
# list(aggs.keys())

@interact(Field=fields)

def viz(Field):
    ihistogram(daily_data, Field, title=f'Distribution of {Field} Field')

interactive(children=(Dropdown(description='Field', options=('Quantity', 'UnitPrice', 'Revenue'), value='Quant…

### Create an interactive line chart where you can explore Quantity and Revenue by day (date) for each country. Include two dropdown widgets - one to choose between the Quantity and Revenue fields and the other to choose the Country.

In [26]:
countries = sorted(data['Country'].unique())
aggregated = data.groupby(['date', 'Country']).agg(aggs).reset_index()
aggregated.head()

,date,Country,Quantity,Revenue,UnitPrice
0,2010-12-01,Australia,107,358.25,5.278571
1,2010-12-01,EIRE,242,505.38,4.182000
2,2010-12-01,France,446,801.86,1.962632
3,2010-12-01,Germany,156,243.48,1.883571
4,2010-12-01,Netherlands,96,177.60,1.850000


In [24]:
def ilinechart(df, x, y, groups=None, title=''):
    fig = px.line(df, x=x, y=y, color=groups, title=title, 
                  template='none').update(layout=dict(title=dict(x=0.5)))

    fig.show()

In [29]:
@interact(Field=['Quantity', 'Revenue'], Country=countries)

def viz(Field, Country):
    filtered = aggregated[aggregated['Country']==Country]
    ilinechart(filtered, 'date', Field, title=f'Daily {Field} for {Country}')

interactive(children=(Dropdown(description='Field', options=('Quantity', 'Revenue'), value='Quantity'), Dropdo…

### Create an interactive scatter plot showing the relationships between daily Quantity, UnitPrice, and Revenue for the United Kingdom. Include two dropdown boxes that let you choose between the 3 fields - one for the x axis and one for the y axis of your scatter plot. Size the data points according to Revenue.

In [30]:
uk_df = data[data['Country']=='United Kingdom']
aggregated = uk_df.groupby('date').agg(aggs).reset_index()
aggregated.head()

,date,Quantity,Revenue,UnitPrice
0,2010-12-01,21308,42213.58,3.172830
1,2010-12-02,30987,47123.68,3.187544
2,2010-12-03,7646,18833.99,3.143373
3,2010-12-05,13603,25807.89,2.818036
4,2010-12-06,15515,29651.87,2.875897


In [31]:
def iscatter(df, x, y, color=None, size=None, title=''):
    fig = px.scatter(df, x=x, y=y, color=color, size=size, 
                     title=title, template='none')

    fig.update_traces(marker_line_color='black', 
                  marker_line_width=1)

    fig.show()

In [35]:
@interact(X_Axis=sorted(fields), Y_Axis=sorted(fields, reverse=True), Size=sorted(fields))

def viz(X_Axis, Y_Axis, Size):
    iscatter(aggregated, X_Axis, Y_Axis, size=Size, title=f'Relationship between {X_Axis} and {Y_Axis}')

interactive(children=(Dropdown(description='X_Axis', options=('Quantity', 'Revenue', 'UnitPrice'), value='Quan…

### Create a bar chart showing the top X products sold (by quantity) in the United Kingdom in a specific month. Use a dropdown box to select the month name and a slider to show the top X products. The range for X should be from 5 to 25. Make sure the bars are sorted in descending order according to their total quantity sold for the month.

Bonus points if you can figure out how to ensure that the month names in the dropdown appear in their correct order (January, February, March, April, etc.).

In [37]:
data['month'] = data['InvoiceDate'].dt.month_name()
data['month_num'] = data['InvoiceDate'].dt.month

aggregated = data.groupby(['month_num', 'month', 'Description']).agg(aggs).reset_index()

aggregated.head()

,month_num,month,Description,Quantity,Revenue,UnitPrice
0,1,January,4 PURPLE FLOCK DINNER CANDLES,2,5.10,2.550000
1,1,January,OVAL WALL MIRROR DIAMANTE,12,119.40,9.950000
2,1,January,SET 2 TEA TOWELS I LOVE LONDON,417,1110.15,2.916667
3,1,January,10 COLOUR SPACEBOY PEN,276,234.60,0.850000
4,1,January,12 COLOURED PARTY BALLOONS,80,52.00,0.650000


In [47]:
def ibarchart(df, x, y, order=None, title=''):
    fig = px.bar(df, x=x, y=y, title=title, template='none')

    fig.update_traces(marker_color='lightskyblue',
                      marker_line_color='black', 
                      marker_line_width=1)
    
    fig.update_layout(xaxis={'categoryorder':'array', 
                             'categoryarray': order})
    
    fig.show()

In [51]:
months = list(aggregated['month'].unique())

@interact(Month=months, Top=(5, 25))

def viz(Month, Top):
    
    monthly = aggregated[aggregated['month']==Month]
    top_prod = monthly.sort_values('Quantity', ascending=False).head(Top)
    ibarchart(top_prod, 'Description', 'Quantity', title=f'Top {Top} Products Sold in {Month}')

interactive(children=(Dropdown(description='Month', options=('January', 'February', 'March', 'April', 'May', '…